In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import progressbar as progressbar

%load_ext autoreload
%autoreload 2

In [2]:
print(f'TF version: {tf.__version__}')
# print(tf.config.list_physical_devices())

TF version: 2.0.0


# Read Data

In [3]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '../datasets/'

## Amazon Fashion

In [4]:
data_path = 'data/Amazon/'
# file_name = 'Amazon_full' # file_name = 'Amazon_05_users' 
# file_name = 'Amazon_01_users'
file_name = 'am_80k_users'

## MovieLens

In [5]:
# data_path = 'data/ML/'
file_name = 'ml_1m'
# file_name = 'ML_full' # file_name = 'ML_05_users'
# file_name = 'ML_01_users'

In [10]:
df = pd.read_pickle(path + file_name)
df.user_id = df.user_id.astype('category').cat.codes
df.item_id = df.item_id.astype('category').cat.codes
df.head()

,user,item,datetime,rating,user_id,item_id
6904244,A2EQZT4NOBKME3,B00SFLJZ52,2015-06-11,3.0,29949,137482
4472551,A2EQZT4NOBKME3,B00D4TJRWG,2015-06-11,5.0,29949,73746
10630561,A2EQZT4NOBKME3,B00OBT081W,2016-01-15,5.0,29949,126216
5801430,A2EQZT4NOBKME3,B00KD9AGAC,2016-08-24,5.0,29949,108075
5505899,A1QKA075BTCNIH,B00ISY7VNO,2015-01-13,5.0,15581,100488


In [9]:
test_set = pd.DataFrame()
train_set = pd.DataFrame()
pbar = progressbar.ProgressBar()
for u in pbar(df.user_id.unique()):
    last_item = df[df.user_id==u].iloc[-1:]
    test_set = test_set.append(last_item)
    remaining = df[df.user_id==u].iloc[:-1]
    train_set = pd.concat([train_set, remaining])

KeyboardInterrupt: 

In [11]:
from Data_prep import leave_users_out
remaining, subset = leave_users_out(df, 40000)

In [12]:
len(subset)

351771

In [13]:
df = subset
df.user_id = df.user_id.astype('category').cat.codes
df.item_id = df.item_id.astype('category').cat.codes

---
# Data Prep

## Dataset Params

In [14]:
val_perc = test_perc = 0.1
n_last_items_val = n_last_items_test = 1

total_items = len(df.item_id.unique())
total_users = len(df.user_id.unique())

## Create Split

In [15]:
from Data_prep import train_val_test_split
datasets = train_val_test_split(df, val_perc, test_perc, n_last_items_val, n_last_items_test)
train_set, val_set, test_set = datasets

---
# Neural Collaborative Filtering (NCF)
Using the NCF framework we build Generalized Matrix Factorisation (GMF), Multiplayer Perceptron Matrix Factorisation (MLP) and combine the two in Neural Matrix Factorisation (NeuMF)
- paper: http://papers.www2017.com.au.s3-website-ap-southeast-2.amazonaws.com/proceedings/p173.pdf
- blog: https://medium.com/@victorkohler/collaborative-filtering-using-deep-neural-networks-in-tensorflow-96e5d41a39a1
- code: https://github.com/Leavingseason/NeuralCF/blob/master

## Set Parameters

### CKPTS Store Paths

In [20]:
run_num = 'am_40k_nolf_8'

In [21]:
GMF_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'nolf': 8,
    'regs': [0,0],
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/GMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

In [22]:
MLP_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'layers': [64,32,16,8],
    'reg_layers': [0,0,0,0],
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/MLP_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

In [23]:
NeuMF_params = {
    'learning_rate': 0.0005,
    'batch_size': 256,
    'layers': [64,32,16,8],
    'reg_layers': [0,0,0,0],
    'reg_mf': [0, 0],
    'nolf': 8,
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/NeuMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

## Init

In [24]:
from NCF import NCF
NCF = NCF(total_users, total_items, GMF_params, MLP_params, NeuMF_params)

NCF.build_GMF_model()
NCF.build_MLP_model()
NCF.build_NeuMF_model()

## Create Samples (MP)

In [25]:
import multiprocessing as mp        
def create_sample(user_items, train_users, train_items, params, num_processes):
    samples_sizes_split = np.array_split(np.array(range(params['sample_size'])),num_processes)
    args = []
    for samples_size in samples_sizes_split:
        args.append((user_items, train_users, train_items, len(samples_size), params['num_neg']))
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(create_sample_worker, args)

    user_inputs, item_inputs, labels = [], [], []
    for res_epochs in results: 
        user_inputs.extend(res_epochs['u'])
        item_inputs.extend(res_epochs['i'])
        labels.extend(res_epochs['l'])

    return user_inputs, item_inputs, labels


def create_sample_worker(user_items, train_users, train_items, sample_size, num_neg):
    user_inputs, item_inputs, labels = [], [], []
    for s in range(sample_size):
        # Add positive item
        u = np.random.choice(train_users)
        u_items = user_items[u]
        i = np.random.choice(u_items)

        user_inputs.append(u)
        item_inputs.append(i)
        labels.append(1)

        # Add negative item
        for i in range(num_neg):
            j = np.random.choice(train_items)
            while j in u_items:  # neg item j cannot be in the set of pos items of user u
                j = np.random.choice(train_items)

            user_inputs.append(u)
            item_inputs.append(j)
            labels.append(0)

    return {'u':user_inputs, 'i':item_inputs, 'l':labels}

In [27]:
%%time
import csv
num_processes = mp.cpu_count()
user_items = train_set.groupby('user_id')['item_id'].apply(list)
train_users = train_set.user_id.unique()
train_items = train_set.item_id.unique()
num_processes = mp.cpu_count()
val_metrics = []
for epoch in range(20):
    print(f'Epoch: {epoch}')
    user_inputs, item_inputs, labels = create_sample(user_items, train_users, train_items, NCF.NeuMF_params, num_processes)
    samples = [user_inputs, item_inputs, labels]
    file = open(f'../NeuMF_storage/samples_am_40k/am_40k_sample_{epoch}.csv', 'w+', newline='')
    with file:
        write = csv.writer(file)
        write.writerows(samples)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
CPU times: user 1min 20s, sys: 8.22 s, total: 1min 28s
Wall time: 3min 59s


In [43]:
import csv

file = open(f'../NeuMF_storage/samples/ml_1m_sample_test.csv', 'w+', newline='')

with file:
    write = csv.writer(file)
    write.writerows(samples)

## Read Sample

In [37]:
%%time
with open('samples.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    samples = list(csv_reader)

CPU times: user 1.01 s, sys: 330 ms, total: 1.34 s
Wall time: 1.34 s
